## EDA & Viz- Multiple datasets - Olist Brazilian E-Commerce Dataset

### Intro

The goal of this project is to extract insights from Olist's e-commerce datasets about brazilian stores, customers and purchased items.

This is the [Kaggle's dataset](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?datasetId=55151&sortBy=voteCount).

Olist has released a dataset with 100k orders made between 2016 and 2018. Each order has some information about the customer and their orders.

### Goals

- Identify top states related to payment value;
- Identify states with the bigest and lowest average ticket;
- Identify most/less selled product categories;
- Identify most/less profitable product categories;
- Identify categories with the bigest average ticket;
- Identify correlations between order price and other features.

### Getting started

Basics firsts (libraries and importing datasets)...

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import altair as alt

plt.style.use("ggplot")
%load_ext lab_black

In [ ]:
df_order = pd.read_csv("olist_orders_dataset.csv")
df_items = pd.read_csv("olist_order_items_dataset.csv")
df_payments = pd.read_csv("olist_order_payments_dataset.csv")
df_products = pd.read_csv("olist_products_dataset.csv")
df_customers = pd.read_csv("olist_customers_dataset.csv")
df_translate = pd.read_csv("product_category_name_translation (1).csv")

Taking a look at the datasets...

In [ ]:
df_order.head()

In [ ]:
df_items.head()

In [ ]:
df_payments.head()

In [ ]:
df_products.head()

In [ ]:
df_customers.head()

In [ ]:
# Joining the data I want and cleaning the output dataset. Note the order change to further easy interpretation (in my opinion).

df = (
    df_order.merge(
        df_customers[["customer_id", "customer_city", "customer_state"]],
        on="customer_id",
        how="left",
    )
    .merge(
        df_items[["order_id", "product_id", "order_item_id", "price"]],
        on="order_id",
        how="left",
    )
    .merge(
        df_products[["product_id", "product_category_name"]],
        on="product_id",
        how="left",
    )
    .merge(
        df_payments[
            ["order_id", "payment_type", "payment_installments", "payment_value"]
        ],
        on="order_id",
        how="left",
    )
    .merge(df_translate, on="product_category_name", how="left")
    .drop(
        columns=[
            "product_category_name",
            "order_status",
            "order_approved_at",
            "order_delivered_carrier_date",
            "order_delivered_customer_date",
            "order_estimated_delivery_date",
        ]
    )
)

df.head()

In [ ]:
df.info()

We have some NaN values here. For simplification, I decided to drop the NaN raws since they represent less than 5% of the dataset.

In [ ]:
df = df.dropna()
df.info()

I'd like to make some further analysis that will use date and time features, so i'll use convert the column `order_purchase_timestamp`.

In [ ]:
df[["order_purchase_timestamp",]] = df[["order_purchase_timestamp",]].apply(
    pd.to_datetime
)
# df [['x','y','z']] = df [['order_purchase_timestamp','order_delivered_customer_date','order_delivered_customer_date']].apply(pd.to_datetime)
df.info()

In [ ]:
# Creating periods of the day classification for purchase time

df["purchase_time_class"] = df["order_purchase_timestamp"].apply(
    lambda x: "morning"
    if (x.hour < 12 and x.hour >= 6)
    else (
        "afternoon"
        if (x.hour < 18 and x.hour >= 12)
        else ("night" if (x.hour < 24 and x.hour >= 18) else "dawn")
    )
)

In [ ]:
# Creating seasons classificantion for purchase time.
# PS: That's brazilian's season and it will be a proxy for simplification purposes.

import datetime

df["year_season"] = df["order_purchase_timestamp"].apply(
    lambda x: "summer"
    if x.month <= 3 and x.month >= 1
    else (
        "spring"
        if x.month >= 10 and x.month <= 12
        else ("winter" if x.month >= 7 and x.month <= 9 else "autumn")
    )
)

In [ ]:
df.head()

So, let's remember what was done so far:

- Imported libraries and datasets;
- Created a final dataset by joining useful information from the existing datasets;
- Removed columns that will not be used;
- Ordered the final dataset to be more interpretable;
- Removed rows with NaN values;
- Converted dates to datetime type;
- And classified the orders by period of the day and year that happened.

### Exploring the features

Finally, after the data preparation we can start our analysis.

I'll take a look at the features.

In [ ]:
plt.figure(figsize=(25, 15))

fig1 = px.histogram(
    df, x="customer_state", y="payment_value", barmode="group", color="payment_type",
).update_xaxes(categoryorder="total descending")
fig1.show()

Wow, it looks like that most of the value comes from SP.

Let's take a closer look at the 5 top states.

In [ ]:
# Creating a list of top 5 and a boxplot to them
list_5 = list(
    (
        df.groupby("customer_state")
        .sum()
        .sort_values("payment_value", ascending=False)
        .reset_index()["customer_state"][:5]
    )
)
plt.figure(figsize=(20, 10))
sns.set(style="darkgrid")
sns.boxplot(
    data=df[df["customer_state"].isin(list_5)],
    x="customer_state",
    y="payment_value",
    palette="dark",
    order=list_5,
)
sns.despine(left=True)

That's not so clarifying, but it's possible o identify some outliers that puts SP and RJ way above the others states. How are these states cities doing?

In [ ]:
alt.Chart(
    df[(df["customer_state"] == "SP") | (df["customer_state"] == "RJ")]
    .groupby("customer_city")
    .sum()
    .sort_values("payment_value", ascending=False)
    .reset_index()[:10]
).mark_bar().encode(alt.X("customer_city", sort="-y"), y="payment_value").properties(
    width=800, height=300
)

As expected, the major cities of the states are also the ones which has spent more money.

Moving on... Let's see which states has the higher and de lesser average tickets.

In [ ]:
# Creating a new dataset to calculate the average ticket
df_tm_state = pd.concat(
    [
        df.groupby("customer_state")
        .count()
        .sort_values("customer_state")
        .reset_index()[["customer_state", "order_id"]],
        df.groupby("customer_state")
        .sum()
        .sort_values("customer_state")
        .reset_index()["payment_value"],
    ],
    axis=1,
)

df_tm_state["average_ticket"] = df_tm_state["payment_value"] / (df_tm_state["order_id"])

# Using Altair to plot a simple graph to plot top 10

alt.Chart(
    df_tm_state.sort_values("average_ticket", ascending=False)
    .reset_index()
    .drop(columns="index")[:10]
).mark_bar().encode(
    alt.X("customer_state", sort="-y"),
    y="average_ticket",
    tooltip=[alt.Tooltip("payment_value"), alt.Tooltip("order_id")],
).properties(
    width=1000, height=500
)

Here we have an interesting information. None of the top states we saw before are among the higher average ticket. 
Since this states are bit far away from the logistics center, maybe the freight price pulled up the total payment value. And the installments may be a part of it too...

On the other hand...

In [ ]:
# Using Altair to plot a simple graph to plot last 10
alt.Chart(
    df_tm_state.sort_values("average_ticket", ascending=False)
    .reset_index()
    .drop(columns="index")[-10:]
).mark_bar().encode(
    alt.X("customer_state", sort="-y"),
    y="average_ticket",
    tooltip=[alt.Tooltip("payment_value"), alt.Tooltip("order_id")],
).properties(
    width=1000, height=500
)

SP, RJ, MG, etc, appears here.

So, although they have a higher total payment value, their average ticket are lower. Again, since they're in the central region of the country and concentrates most of logistics infraestructure, maybe there is no freight to raise the payment value.


Moving on to the product categories...

In [ ]:
# Creating a top ten product categories bar plot

sns.set(style="darkgrid")
with sns.plotting_context("notebook", font_scale=3):
    sns.catplot(
        x="product_category_name_english",
        y="payment_value",
        kind="bar",
        data=df.groupby("product_category_name_english")
        .sum()
        .sort_values("payment_value", ascending=False)
        .reset_index()[:10],
        height=20,
        aspect=2,
        color="green",
    )
# plt.figure(figsize=(20, 10))
plt.xticks(rotation=45)

As we can see, the highlight is the top 6 categories. Let's take a look at the other side. 

In [ ]:
sns.set(style="darkgrid")
with sns.plotting_context("notebook", font_scale=3):
    sns.catplot(
        x="product_category_name_english",
        y="payment_value",
        kind="bar",
        data=df.groupby("product_category_name_english")
        .sum()
        .sort_values("payment_value", ascending=False)
        .reset_index()[-10:],
        height=20,
        aspect=2,
        color="green",
    )
# plt.figure(figsize=(20, 10))
plt.xticks(rotation=45)

The 10 categories with lower payment value are under 5.000 and the last 2 ones are under 1.000. There is a huge diference between categories then.

And what about the amount of items sold?

In [ ]:
upper = (
    alt.Chart(
        df.groupby("product_category_name_english")
        .count()
        .sort_values("order_id", ascending=False)
        .reset_index()
        .rename(columns={"order_id": ("items_sold")})[
            ["product_category_name_english", "items_sold"]
        ][:10]
    )
    .mark_bar()
    .encode(alt.X("product_category_name_english", sort="-y"), y="items_sold",)
    .properties(width=1200, height=400)
)

lower = (
    alt.Chart(
        df.groupby("product_category_name_english")
        .count()
        .sort_values("order_id", ascending=False)
        .reset_index()
        .rename(columns={"order_id": ("items_sold")})[
            ["product_category_name_english", "items_sold"]
        ][-10:]
    )
    .mark_bar()
    .encode(alt.X("product_category_name_english", sort="-y"), y="items_sold",)
    .properties(width=1200, height=400)
)

upper & lower

Again, a huge diference. And it seems like to corroborate with our last analysis, since the same categories appears on both graphs.
For instance, `bed_bath_table` is top 1 on both.

Moving on to analyzing the average ticket of categories.

In [ ]:
# Creating a new dataset for the average ticket
df_tm_categories = pd.concat(
    [
        df.groupby("product_category_name_english")
        .count()
        .sort_values("product_category_name_english")
        .reset_index()[["product_category_name_english", "order_id"]],
        df.groupby("product_category_name_english")
        .sum()
        .sort_values("product_category_name_english")
        .reset_index()["payment_value"],
    ],
    axis=1,
)

df_tm_categories["average_ticket"] = df_tm_categories["payment_value"] / (
    df_tm_categories["order_id"]
)

# Implementing a two rows plot with the first 10 and last 10 average tickets
upper = (
    alt.Chart(
        df_tm_categories.sort_values("average_ticket", ascending=False)
        .reset_index()
        .drop(columns="index")[:10]
    )
    .mark_bar()
    .encode(
        alt.X("product_category_name_english", sort="-y"),
        y="average_ticket",
        tooltip=[alt.Tooltip("average_ticket")],
    )
    .properties(width=1200, height=400)
)

lower = (
    alt.Chart(
        df_tm_categories.sort_values("average_ticket", ascending=False)
        .reset_index()
        .drop(columns="index")[-10:]
    )
    .mark_bar()
    .encode(
        alt.X("product_category_name_english", sort="-y"),
        y="average_ticket",
        tooltip=[alt.Tooltip("average_ticket")],
    )
    .properties(width=1200, height=400)
)

upper & lower

The computer category stands out from the rest and it makes sense, since computers have high added value.

On the other hand, home confort and flowers have lower added value, so, naturally, their average ticket are the lowest ones (under 70).

### Feature relations

Now, we're going to analyze the relationship between variables.

Are there correlation between price and time of buying? And what about the season?
Does the credit card are related to higher value orders?

LET'S CHECK OUT!

In [ ]:
alt.data_transformers.disable_max_rows()

alt.Chart(df).mark_point().encode(
    x=alt.X(
        field="year_season",
        type="nominal",
        sort=["spring", "summer", "autumn", "winter"],
    ),
    y="payment_value:Q",
    color="payment_type:N",
    column="payment_type:N",
).properties(width=300, height=250)

In [ ]:
alt.Chart(df).mark_point().encode(
    x=alt.X(
        field="purchase_time_class",
        type="nominal",
        sort=["morning", "afternoon", "night", "dawn"],
    ),
    y="payment_value:Q",
    color="payment_type:N",
    column="payment_type:N",
).properties(width=300, height=250)

Although we have so many outliers, its possible to tell that there's no easy noticeable correlation. With the exception of dawn, that has fewer orders, the other periods of the day has similar buying behavior.

It seems like that on autumn and winter the overall payment value is a little bit higher. And, visually, it's possible to see that apparently the majority of the orders are paid with credit card, the purchase time is more often afternoon and night and at summer and autumn.

In [ ]:
# Creating a mean with error whiskers plot to see the overall trend
plt.figure(figsize=(20, 10))
sns.pointplot(
    x="purchase_time_class", y="payment_value", hue="payment_type", data=df, dodge=True
)

In [ ]:
plt.figure(figsize=(20, 10))
sns.pointplot(
    x="year_season",
    y="payment_value",
    hue="payment_type",
    order=["spring", "summer", "autumn", "winter"],
    data=df,
    dodge=True,
)

It confirms what we suspected before. No correlations screaming on the screen.

### Conclusion

So, at the end, we were able to achieve our initial goals.

It is importante to say that the OLIST datasets has way more features and many aspects to study than I explored here. I decided to try to explore categorical features (even created ones), 'cause I want to practice the visualiton and libraries.

Feel free to contribute. I'm a learner.

That's all folks!